<a href="https://colab.research.google.com/github/RafsanJany-44/Thesis_Project/blob/main/SignalReading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, find_events, pick_types, set_eeg_reference
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from mne import viz

%matplotlib inline

In [ ]:
# The eeg data contains four event markers. We'll use these to explore the data.
event_id = dict(response=4096, target=8192, NonTarget_High=16384, NonTarget_Low=32768)

# This will determine the time window for the data that we care about
tmin, tmax = -0.1, 1.

# path to data (change this to whereever you saved your data)
myPath = r'C:\Users\seth\Documents\dev\Hackathon\brain_sample'

In [ ]:
# multiple subjects

# raw_fnames = eegbci.load_data(subject, runs)
# raw_fnames = [os.path.join(myPath,f) for f in os.listdir(myPath)]
# raw_files = [read_raw_edf(f, preload=True, stim_channel='auto', verbose=False) for f in
#              raw_fnames]
# raw = concatenate_raws(raw_files, verbose=False)

#2) Load our EEG data¶

In [ ]:
# single subject
data = os.listdir(myPath)
dataPath = os.path.join(myPath, data[0])

# We use mne.read_raw_edf to read in the .bdf EEG files
raw = read_raw_edf(dataPath, preload=True, stim_channel='auto', verbose=False)
raw

#Let's explore our EEG data!

In [ ]:
# what does our data look like?

print('Data type: {}\n\n{}\n'.format(type(raw), raw))
# print('{} \n'.format(raw))

# Get the sample rate
print('Sample rate:', raw.info['sfreq'], 'Hz')

# Get the size of the matrix
print('Size of the matrix: {}\n'.format(raw.get_data().shape))

# We can use the mne.info class to learn moure about the data.
print(raw.info)

In [ ]:
print('The actual data is just a matrix array!\n\n {}\n'.format(raw.get_data()))

#Start to visualize the data

In [ ]:
# Extract data from the first 5 channels, from 1 s to 3 s.
sfreq = raw.info['sfreq']
sfreq = raw.info['sfreq']
data, times = raw[:5, int(sfreq * 1):int(sfreq * 10)]

fig = plt.subplots(figsize=(10,8))
plt.plot(times, data.T);
plt.xlabel('Seconds')
plt.ylabel('$\mu V$')
plt.title('Channels: 1-5');
plt.legend(raw.ch_names[:5]);